# Module 11 - Programming Assignment

## Directions

1. Change the name of this file to be your JHED id as in `jsmith299.ipynb`. Because sure you use your JHED ID (it's made out of your name and not your student id which is just letters and numbers).
2. Make sure the notebook you submit is cleanly and fully executed. I do not grade unexecuted notebooks.
3. Submit your notebook back in Blackboard where you downloaded this file.

*Provide the output **exactly** as requested*

In [1]:
from copy import deepcopy
import numpy as np

## Reinforcement Learning with Value Iteration

These are the same maps from Module 1 but the "physics" of the world have changed. In Module 1, the world was deterministic. When the agent moved "south", it went "south". When it moved "east", it went "east". Now, the agent only succeeds in going where it wants to go *sometimes*. There is a probability distribution over the possible states so that when the agent moves "south", there is a small probability that it will go "east", "north", or "west" instead and have to move from there.

There are a variety of ways to handle this problem. For example, if using A\* search, if the agent finds itself off the solution, you can simply calculate a new solution from where the agent ended up. Although this sounds like a really bad idea, it has actually been shown to work really well in video games that use formal planning algorithms (which we will cover later). When these algorithms were first designed, this was unthinkable. Thank you, Moore's Law!

Another approach is to use Reinforcement Learning which covers problems where there is some kind of general uncertainty in the actions. We're going to model that uncertainty a bit unrealistically here but it'll show you how the algorithm works.

As far as RL is concerned, there are a variety of options there: model-based and model-free, Value Iteration, Q-Learning and SARSA. You are going to use Value Iteration.

## The World Representation

As before, we're going to simplify the problem by working in a grid world. The symbols that form the grid have a special meaning as they specify the type of the terrain and the cost to enter a grid cell with that type of terrain:

```
token   terrain    cost 
.       plains     1
*       forest     3
^       hills      5
~       swamp      7
x       mountains  impassible
```

When you go from a plains node to a forest node it costs 3. When you go from a forest node to a plains node, it costs 1. You can think of the grid as a big graph. Each grid cell (terrain symbol) is a node and there are edges to the north, south, east and west (except at the edges).

There are quite a few differences between A\* Search and Reinforcement Learning but one of the most salient is that A\* Search returns a plan of N steps that gets us from A to Z, for example, A->C->E->G.... Reinforcement Learning, on the other hand, returns  a *policy* that tells us the best thing to do in **every state.**

For example, the policy might say that the best thing to do in A is go to C. However, we might find ourselves in D instead. But the policy covers this possibility, it might say, D->E. Trying this action might land us in C and the policy will say, C->E, etc. At least with offline learning, everything will be learned in advance (in online learning, you can only learn by doing and so you may act according to a known but suboptimal policy).

Nevertheless, if you were asked for a "best case" plan from (0, 0) to (n-1, n-1), you could (and will) be able to read it off the policy because there is a best action for every state. You will be asked to provide this in your assignment.

We have the same costs as before. Note that we've negated them this time because RL requires negative costs and positive rewards:

In [2]:
costs = { '.': -1, '*': -3, '^': -5, '~': -7}
costs

{'.': -1, '*': -3, '^': -5, '~': -7}

and a list of offsets for `cardinal_moves`. You'll need to work this into your **actions**, A, parameter.

In [3]:
cardinal_moves = [(1,0), (0,-1), (-1,0), (0,1)]

For Value Iteration, we require knowledge of the *transition* function, as a probability distribution.

The transition function, T, for this problem is 0.70 for the desired direction, and 0.10 each for the other possible directions. That is, if the agent selects "north" then 70% of the time, it will go "north" but 10% of the time it will go "east", 10% of the time it will go "west", and 10% of the time it will go "south". If agent is at the edge of the map, it simply bounces back to the current state.

You need to implement `value_iteration()` with the following parameters:

+ world: a `List` of `List`s of terrain (this is S from S, A, T, gamma, R)
+ costs: a `Dict` of costs by terrain (this is part of R)
+ goal: A `Tuple` of (x, y) stating the goal state.
+ reward: The reward for achieving the goal state.
+ actions: a `List` of possible actions, A, as offsets.
+ gamma: the discount rate

you will return a policy: 

`{(x1, y1): action1, (x2, y2): action2, ...}`

Remember...a policy is what to do in any state for all the states. Notice how this is different than A\* search which only returns actions to take from the start to the goal. This also explains why reinforcement learning doesn't take a `start` state.

You should also define a function `pretty_print_policy( cols, rows, policy)` that takes a policy and prints it out as a grid using "^" for up, "<" for left, "v" for down and ">" for right. Use "x" for any mountain or other impassable square. Note that it doesn't need the `world` because the policy has a move for every state. However, you do need to know how big the grid is so you can pull the values out of the `Dict` that is returned.

```
vvvvvvv
vvvvvvv
vvvvvvv
>>>>>>v
^^^>>>v
^^^>>>v
^^^>>>G
```

(Note that that policy is completely made up and only illustrative of the desired output). Please print it out exactly as requested: **NO EXTRA SPACES OR LINES**.

* If everything is otherwise the same, do you think that the path from (0,0) to the goal would be the same for both A\* Search and Q-Learning?
* What do you think if you have a map that looks like:

```
><>>^
>>>>v
>>>>v
>>>>v
>>>>G
```

has this converged? Is this a "correct" policy? What are the problems with this policy as it is?


The path form the A* search and the Q-Learning would be same because both algorithms give the most optimal paths. Assuming there is only one optimal path.<br>
The policy shown above is incomplete because there are cycles, and ther is a state on the top right corner that points outside the world. I think the above policy has not conveged, we should either let the algorithm run for some more iterations or increase the reward for the goal.

In [4]:
def read_world(filename):
    result = []
    with open(filename) as f:
        for line in f.readlines():
            if len(line) > 0:
                result.append(list(line.strip()))
    return result

---

**is_valid**<br>
The `is_valid` is a helper functionf for `value_iteration`. It checks if the given action is in the boundry of the world.

Parameters:
* **world**: a `List` of `List`s of terrain (this is S from S, A, T, gamma, R)
* **s** is the `(col, row)` pair for the current state being scanned.
* **a** is the `(col, row)` pair that we want to check if its valid.


The function returns `True` if action is valud else it returns `False`</br>
For example if the world, s and a as shown below
```
w = [['.', '.', '.', '.', '.', '.'],
     ['.', '*', '*', '*', '*', '.'],
     ['.', '*', '*', '*', '*', '.'],
     ['.', '*', '*', 'x', '*', '.'],
     ['.', '*', '*', '*', '*', '.'],
     ['.', '.', '.', '.', '.', '.'],
     ['.', '.', '.', '.', '.', '.']]
     
s = (0,0)
a = (0,-1)
```
retuns:<br>
`False`

In [5]:
def is_valid(world, s, a):
    row = len(world)
    col = len(world[0])
    
    if (s[0]+a[0]>=0 and s[0]+a[0]<row) and (s[1]+a[1]>=0 and s[1]+a[1]<col):
        if world[s[0]+a[0]][s[1]+a[1]] == 'x':
            return False
        return True
    return False

**calc_reward**<br>
The `calc_reward` is a helper functionf for `value_iteration`. It calculates the states.

Parameters:
* **world**: a `List` of `List`s of terrain (this is S from S, A, T, gamma, R)
* **s** is the `(col, row)` pair for the current state being scanned.
* **a** is the `(col, row)` pair that we want to check if its valid.
* **costs**: a `Dict` of costs by terrain (this is part of R)
* **goal**: A `Tuple` of (x, y) stating the goal state.


The function returns `cost` of the terrain. </br>
For example if the world, s and a, rewards as shown below, and costs as shown above
```
w = [['.', '.', '.', '.', '.', '.'],
     ['.', '*', '*', '*', '*', '.'],
     ['.', '*', '*', '*', '*', '.'],
     ['.', '*', '*', 'x', '*', '.'],
     ['.', '*', '*', '*', '*', '.'],
     ['.', '.', '.', '.', '.', '.'],
     ['.', '.', '.', '.', '.', '.']]
     
s = (0,0)
a = (0,1)
```
retuns:<br>
`-1`

In [6]:
def calc_reward(world, s, a, costs, rewards, goal):
    if s[0]+a[0] == goal[1] and s[1]+a[1] == goal[0]:
        return rewards + costs[world[s[0]+a[0]][s[1]+a[1]]]
    terrain = world[s[0]+a[0]][s[1]+a[1]]
    t_cost = costs[terrain] if terrain in costs else -100
    return t_cost

**transition**<br>
The `transition` is a helper functionf for `value_iteration`. it calculates the transition 
reward for the state.

Parameters:
* **world**: a `List` of `List`s of terrain (this is S from S, A, T, gamma, R)
* **s** is the `(col, row)` pair for the current state being scanned.
* **planned_a** is the `(col, row)` pair that states the planned action.
* **actions**: a `List` of possible actions, A, as offsets.
* **costs**: a `Dict` of costs by terrain (this is part of R)
* **goal**: A `Tuple` of (x, y) stating the goal state.
* **vs_last**: a `List` of `List`s of rewards from previous states
* **pp**: desired(planned) probability. It is set to 70% by default
* **sp**: surporise probability for non desired direction. It is set to 10% by default.

The function returns `transition reward` of the state. </br>

In [7]:
def transition(world, s, planned_a, actions, vs_last, pp = 0.70, sp = 0.10):
    tmp_sum = 0
    for a in actions:
        if is_valid(world, s, a):
            if a == planned_a:
                tmp_sum += pp* vs_last[s[0]+a[0]][s[1]+a[1]]
            else:
                tmp_sum += sp* vs_last[s[0]+a[0]][s[1]+a[1]]
    return tmp_sum

**value_iteration**<br>
The `value_iteration` is the value iteration (stocastic version) algorithm implementation.

Parameters:
* **world**: a `List` of `List`s of terrain (this is S from S, A, T, gamma, R)
* **actions**: a `List` of possible actions, A, as offsets.
* **costs**: a `Dict` of costs by terrain (this is part of R)
* **goal**: A `Tuple` of (x, y) stating the goal state.

returns<br>
It returns `policy dictionary`</br>

In [8]:
def value_iteration(world, costs, goal, rewards, actions, gamma):
    plc, t, decode_action = {}, 0, {(0,1):">", (1,0):"v", (-1,0):"^", (0,-1):"<"}
    vs = [[0 for _ in range(len(world[0]))] for _ in range(len(world))]
    while True:
        vs_last, t = deepcopy(vs), t+1
        for row in range(len(world)):
            for col in range(len(world[0])):
                if world[row][col] != 'x':
                    qsa = {}
                    for a in actions:
                        if is_valid(world,(row,col), a):        # check if the action is valid
                            qsa[(row,col),decode_action[a]] = calc_reward(world, (row,col), a, costs, rewards, goal) + gamma * transition(world, (row,col), a, actions, vs_last)
                    pi_s = max(qsa, key = lambda x:qsa[x])      # find the action with best reward
                    vs[row][col], plc[(row,col)]  = round(qsa[pi_s],2), pi_s[1]
                else:
                    plc[(row,col)] = 'x'
        if np.max(abs(np.array(vs)-np.array(vs_last))) < 0.01:
            break
    return plc

**pretty_print_policy**<br>
The `pretty_print_policy` is the function that prints the policy.

Parameters:
* **cols**: number of columns in world.
* **rows**: number of rows in world.
* **policy**: a `Dict` of policy.
* **goal**: A `Tuple` of (x, y) stating the goal state.

The function prints the policy in the reqested format.
For example, if policy is as shown below<br>
```
policy = {(0,0):>,
          (0,1):>,
          (0,2):v,
          (1,0):v,
          (1,2):v,
          (1,1):x,
          (2,0):>,
          (2,1):>,
          (2,2):G}
rows = 3
cols = 3
goal = (2,2),
```
returns<br>
```
>>v
vxv
>>G
```

In [9]:
def pretty_print_policy( cols, rows, policy, goal):
    policy[(goal[1], goal[0])] = 'G'
    for row in range(rows):
        tmp_row = [policy[(row,col)] for col in range(cols)]
        print("".join(tmp_row))

## Value Iteration

### Small World

In [10]:
small_world = read_world( "small.txt")

In [11]:
goal = (len(small_world[0])-1, len(small_world)-1)
gamma = 0.9
reward = 10

small_policy = value_iteration(small_world, costs, goal, reward, cardinal_moves, gamma)

In [12]:
cols = len(small_world[0])
rows = len(small_world)

pretty_print_policy(cols, rows, small_policy, goal)

v>>>>v
v<^^>v
v<v>>v
v<vx>v
vvvvvv
>>>>>v
>>>>>G


### Large World

In [13]:
large_world = read_world( "large.txt")

In [ ]:
goal = (len(large_world[0])-1, len(large_world)-1) # Lower Right Corner FILL ME IN
gamma = 0.9
reward = 17500
large_policy = value_iteration(large_world, costs, goal, reward, cardinal_moves, gamma)

In [ ]:
cols = len(large_world[0])
rows = len(large_world)

pretty_print_policy( cols, rows, large_policy, goal)

## Before You Submit...

1. Did you provide output exactly as requested?
2. Did you re-execute the entire notebook? ("Restart Kernel and Rull All Cells...")
3. If you did not complete the assignment or had difficulty please explain what gave you the most difficulty in the Markdown cell below.
4. Did you change the name of the file to `jhed_id.ipynb`?

Do not submit any other files.